# Projeto Banco de Dados
# Ocorrências Policiais de São Paulo/SP

# Objetivo
Com base em informações de Ocorrências Policiais do Estado de São Paulo, entre os anos de 2002 e 2021, pretende-se neste projeto avaliar alguns questionamentos, como:

1. 5 anos com maiores incidentes de homicidio;
2. Top 10 cidades em registros de homicídio nos últimos 10 anos (2011-2021);
3. Top 5 cidades em registros de Roubo e Furto;
4. Histórico dos últimos 5 anos de Roubo e Furtos na cidade de São Paulo;
5. Top 5 anos em quantidade de ocorrências policiais no estado.


# 1. Detalhamento

## Origem dos Dados
Dados públicos fornecidos pela Secretaria de Segurança Pública do Estado de São Paulo

## Catálogo de Dados

1. Tabela de Ocorrências:

![Catalogo Ocorrencias](https://raw.githubusercontent.com/pauloricardofagundes/MVP-Engenharia_Dados/refs/heads/main/Catalogo_dados_MVP1.png)

2. Tabela de Municípios:
- Serão carregados para criação do banco de dados apenas os 2 campos necessários para identificação do município

![Catalogo Municipios](https://raw.githubusercontent.com/pauloricardofagundes/MVP-Engenharia_Dados/refs/heads/main/Catalogo_dados_MVP2.png)

In [0]:
from pyspark import SparkFiles
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number



# 2. Importação dos dados para criação do Database

In [0]:
# Base de dados com registro de ocorrências
url_1 = "https://raw.githubusercontent.com/pauloricardofagundes/MVP-Engenharia_Dados/refs/heads/main/br_sp_gov_ssp_ocorrencias_registradas.csv"  

# Base de identificação de municícipios, para join com a base de ocorrências
url_2 = "https://raw.githubusercontent.com/pauloricardofagundes/MVP-Engenharia_Dados/refs/heads/main/br_bd_diretorios_brasil_municipio.csv"  

spark.sparkContext.addFile(url_1)
df_bd_ocorrencias = spark.read.csv(
    "file://" + SparkFiles.get("br_sp_gov_ssp_ocorrencias_registradas.csv"),
    header=True,
    inferSchema=False,
)

spark.sparkContext.addFile(url_2)
df_bd_municipios = spark.read.csv(
    "file://" + SparkFiles.get("br_bd_diretorios_brasil_municipio.csv"),
    header=True,
    inferSchema=False,
)

# Convertendo campos numéricos de texto para número inteiro
Campos_Num = [  "ano",
                "homicidio_doloso",
                "numero_de_vitimas_em_homicidio_doloso",
                "homicidio_doloso_por_acidente_de_transito",
                "numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito",
                "homicidio_culposo_por_acidente_de_transito",
                "homicidio_culposo_outros",
                "tentativa_de_homicidio",
                "lesao_corporal_seguida_de_morte",
                "lesao_corporal_dolosa",
                "lesao_corporal_culposa_por_acidente_de_transito",
                "lesao_corporal_culposa_outras",
                "latrocinio",
                "numero_de_vitimas_em_latrocinio",
                "total_de_estupro",
                "estupro",
                "estupro_de_vulneravel",
                "total_de_roubo_outros",
                "roubo_outros",
                "roubo_de_veiculo",
                "roubo_a_banco",
                "roubo_de_carga",
                "furto_outros",
                "furto_de_veiculo"
                ]

for x in Campos_Num: 
    df_bd_ocorrencias = df_bd_ocorrencias.withColumn(x, df_bd_ocorrencias[x].cast(IntegerType()))

# Selecionar colunas específicas da tabela de município
colunas_desejadas = ["id_municipio", "nome"]
df_bd_municipios = df_bd_municipios.select(colunas_desejadas)

df_bd_ocorrencias = df_bd_ocorrencias.dropna()  # Deleta linhas sem valores
df_bd_municipios = df_bd_municipios.dropna()  # Deleta linhas sem valores


# 3. Criação do Banco de Dados

In [0]:
%sql DROP DATABASE Db_Ocorrencia CASCADE;


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2543913894165253>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-2543913894165253>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("RFJPUCBEQVRBQkFTRSBEYl9PY29ycmVuY2lhIC0tQ0FTQ0FERTs=").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return 

In [0]:
%sql CREATE DATABASE Db_Ocorrencia;

In [0]:
df_bd_ocorrencias.write.mode("overwrite").saveAsTable("Db_Ocorrencia.ocorrencias")

df_bd_municipios.write.mode("overwrite").saveAsTable("Db_Ocorrencia.municipios")

In [0]:
%sql SELECT * FROM Db_Ocorrencia.ocorrencias LIMIT 5

ano,mes,id_municipio,regiao_ssp,homicidio_doloso,numero_de_vitimas_em_homicidio_doloso,homicidio_doloso_por_acidente_de_transito,numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito,homicidio_culposo_por_acidente_de_transito,homicidio_culposo_outros,tentativa_de_homicidio,lesao_corporal_seguida_de_morte,lesao_corporal_dolosa,lesao_corporal_culposa_por_acidente_de_transito,lesao_corporal_culposa_outras,latrocinio,numero_de_vitimas_em_latrocinio,total_de_estupro,estupro,estupro_de_vulneravel,total_de_roubo_outros,roubo_outros,roubo_de_veiculo,roubo_a_banco,roubo_de_carga,furto_outros,furto_de_veiculo
2020,1,3502754,Sorocaba,0,0,0,0,0,0,0,0,2,0,0,0,0,3,1,2,4,3,0,0,1,15,1
2020,2,3502754,Sorocaba,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,2,0,1,0,2,10,1
2020,3,3502754,Sorocaba,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,5,2,3,0,3,8,0
2020,4,3502754,Sorocaba,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,2,2,1,0,0,12,5
2020,5,3502754,Sorocaba,0,0,0,0,1,0,2,0,2,0,0,0,0,1,1,0,2,1,1,0,1,7,1


In [0]:
%sql SELECT * FROM Db_Ocorrencia.municipios LIMIT 5

id_municipio,nome
1100338,Nova Mamoré
1100205,Porto Velho
1101104,Itapuã do Oeste
1100809,Candeias do Jamari
1100940,Cujubim


# 4. Junção de tabelas para criação de tabela flat final

In [0]:
%sql
SELECT
Id_municipio,
nome,
COUNT(*) as contagem
FROM Db_Ocorrencia.municipios
GROUP BY id_municipio, nome
HAVING COUNT(*) > 1
ORDER BY contagem DESC

Id_municipio,nome,contagem


In [0]:
%sql CREATE TABLE Db_Ocorrencia.Geral AS 
SELECT 
O.* ,
M.nome
FROM Db_Ocorrencia.ocorrencias AS O
 
LEFT JOIN 
 (SELECT DISTINCT
 Id_municipio,
 nome
 From Db_Ocorrencia.municipios ) AS M
ON O.id_municipio = M.id_municipio

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM db_ocorrencia.geral
LIMIT 5

ano,mes,id_municipio,regiao_ssp,homicidio_doloso,numero_de_vitimas_em_homicidio_doloso,homicidio_doloso_por_acidente_de_transito,numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito,homicidio_culposo_por_acidente_de_transito,homicidio_culposo_outros,tentativa_de_homicidio,lesao_corporal_seguida_de_morte,lesao_corporal_dolosa,lesao_corporal_culposa_por_acidente_de_transito,lesao_corporal_culposa_outras,latrocinio,numero_de_vitimas_em_latrocinio,total_de_estupro,estupro,estupro_de_vulneravel,total_de_roubo_outros,roubo_outros,roubo_de_veiculo,roubo_a_banco,roubo_de_carga,furto_outros,furto_de_veiculo,nome
2020,1,3502754,Sorocaba,0,0,0,0,0,0,0,0,2,0,0,0,0,3,1,2,4,3,0,0,1,15,1,Araçariguama
2020,2,3502754,Sorocaba,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,2,0,1,0,2,10,1,Araçariguama
2020,3,3502754,Sorocaba,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,5,2,3,0,3,8,0,Araçariguama
2020,4,3502754,Sorocaba,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,2,2,1,0,0,12,5,Araçariguama
2020,5,3502754,Sorocaba,0,0,0,0,1,0,2,0,2,0,0,0,0,1,1,0,2,1,1,0,1,7,1,Araçariguama


# 5. Análises dos Dados

In [0]:
%sql
SELECT
ano as Ano,
Sum(homicidio_doloso) as Hom_Doloso,
Sum(homicidio_doloso_por_acidente_de_transito) as Hom_Dol_AT,
Sum(homicidio_culposo_outros) as Hom_Cul,
Sum(homicidio_culposo_por_acidente_de_transito) as Hom_Cul_AT,
Sum(homicidio_doloso) + Sum(homicidio_doloso_por_acidente_de_transito) + sum(homicidio_culposo_outros) + sum(homicidio_culposo_por_acidente_de_transito)  as Total,

Round(((
  Sum(homicidio_doloso) +
  Sum(homicidio_doloso_por_acidente_de_transito) + 
  Sum(homicidio_culposo_outros) + 
  Sum(homicidio_culposo_por_acidente_de_transito)) 
  * 100.0 / 
  (SELECT (
    Sum(homicidio_doloso) + 
    Sum(homicidio_doloso_por_acidente_de_transito) + 
    Sum(homicidio_culposo_outros) + 
    Sum(homicidio_culposo_por_acidente_de_transito)) 
    FROM Db_Ocorrencia.geral)),2) AS `% Perc`

FROM Db_Ocorrencia.geral
GROUP BY Ano
ORDER BY Total DESC
LIMIT 5

Ano,Hom_Doloso,Hom_Dol_AT,Hom_Cul,Hom_Cul_AT,Total,% Perc
2017,3289,52,159,3473,6973,23.39
2018,2949,36,152,3193,6330,21.23
2019,2776,18,183,3236,6213,20.84
2020,2893,19,131,3153,6196,20.78
2021,1848,4,97,2156,4105,13.77


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT
nome as Municipio,
Sum(homicidio_doloso) as Hom_Doloso,
Sum(homicidio_doloso_por_acidente_de_transito) as Hom_Dol_AT,
Sum(homicidio_culposo_outros) as Hom_Cul,
Sum(homicidio_culposo_por_acidente_de_transito) as Hom_Cul_AT,
Sum(homicidio_doloso) + Sum(homicidio_doloso_por_acidente_de_transito) + sum(homicidio_culposo_outros) + sum(homicidio_culposo_por_acidente_de_transito)  as Total,

Round(((
  Sum(homicidio_doloso) +
  Sum(homicidio_doloso_por_acidente_de_transito) + 
  Sum(homicidio_culposo_outros) + 
  Sum(homicidio_culposo_por_acidente_de_transito)) 
  * 100.0 / 
  (SELECT (
    Sum(homicidio_doloso) + 
    Sum(homicidio_doloso_por_acidente_de_transito) + 
    Sum(homicidio_culposo_outros) + 
    Sum(homicidio_culposo_por_acidente_de_transito)) 
    FROM Db_Ocorrencia.geral)),2) AS `% Perc`

FROM Db_Ocorrencia.geral
WHERE Ano BETWEEN 2011 and 2021
GROUP BY Municipio
ORDER BY Total DESC
LIMIT 10

Municipio,Hom_Doloso,Hom_Dol_AT,Hom_Cul,Hom_Cul_AT,Total,% Perc
São Paulo,3081,44,125,1847,5097,17.09
Campinas,604,0,12,434,1050,3.52
Guarulhos,414,3,21,384,822,2.76
Sorocaba,227,2,1,245,475,1.59
Ribeirão Preto,208,2,8,219,437,1.47
São José dos Campos,186,1,3,206,396,1.33
São Bernardo do Campo,182,5,13,193,393,1.32
Osasco,240,2,13,128,383,1.28
Jundiaí,82,0,2,225,309,1.04
Santo André,199,1,13,85,298,1.00
